In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
#크롬 브라우저 열기
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='user_id'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'user_pw'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)


In [3]:
def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

In [4]:
def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
time.sleep(10)

In [5]:
tag_list =[]
def get_content(driver):
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    data = [content,tag_data]
    return data

In [6]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

In [10]:
word = "풀무원바른먹거리"
url = insta_searching(word)
driver.implicitly_wait(9)
driver.get(url)
time.sleep(9)
click_first(driver)
result_list=[]
target = 10

for i in range(target):
    try:
        data = get_content(driver)
        result_list.append(data)
        next_page(driver)
    except:
        time.sleep(9)
        next_page(driver)
    

In [11]:
result_list

[['.[풀무원- 바른 먹거리 교육] 3-5학년 엄마들이 좋아하는 교육. 🤗영양소의 종류도 이참에 확실히 알고, 판매되는 제품들의 성분분석도 해 볼 수 있었던 깨알같은 수업~집밥이 제일 건강한 먹거리라는 결론. 🤔😉#0428 #juneha #풀무원바른먹거리 2021.04.21  04.28 🕒',
  ['0428', 'juneha', '풀무원바른먹거리']],
 ['#저녁메뉴...우리집 풀무원사랑💚..#풀무원바른먹거리#푸드머스#스틱피쉬커틀렛',
  ['저녁메뉴', '풀무원바른먹거리', '푸드머스', '스틱피쉬커틀렛']],
 ['💕풀무원 바른 먹거리 온라인 비대면 교육💕1차시/2월17일2차시/2월24일풀무원 바른 먹거리에서 초등학교 3,4,5학년 대상으로 올바른 식습관과 211식사법으로 골고루 잘먹는 영양균형교육 그리고 식품포장지 확인하는법과 식품표시 확인교육을 2차시에 걸쳐서 온라인으로 진행해 주셨어요~^^엄마가 요리해주면 몸에 좋은거다 생각하며 먹는데만 집중했던 딸이 1차시 교육을 통해 탄수화물 단백질 지방 비타민 무기질 5대 영양소에는 어떤 식재료와 식품들이 있는지 구분하는 눈이 생겼고 목이 마르면 마셨던 물은 우리몸에 꼭 필요하며 영양소를 운반하고 체온을 조절해주니 자주 마시겠다고 얘기해주네요~♡2차시 교육에서는 식품을 고를땐 무엇을 보고 선택하는지 알아보고 챌이 준비한 과자봉지를 보며사용된재료와 식품첨가물도 살펴보고 식품만들기가 완성된 제조일자와 식품이 만들어진날부터 판매할수있는날인 유통기한 그리고 식품에 들어있는 영양소가 적힌 영양성분까지 학습할수 있는 유익한 시간이었어요~^^자라나는 아이들에게 올바른 식습관과 바른먹거리를 선택할 수있도록 교육해주신 풀무원 바른먹거리 관계자분들 너무너무 감사합니다.♥️👏👍#풀무원 #풀무원바른먹거리 #바른먹거리교육 #비대면교육 #초등집콕생활 @pulmuonefoundation',
  ['풀무원', '풀무원바른먹거리', '바른먹거리교육', '비대면교육', '초등집콕생활']],
 ['..풀무원 달걀농장 체험중~😘..#오한결#풀

In [12]:
df = pd.DataFrame(result_list)
df

,0,1
0,.[풀무원- 바른 먹거리 교육] 3-5학년 엄마들이 좋아하는 교육. 🤗영양소의 종류...,"[0428, juneha, 풀무원바른먹거리]"
1,#저녁메뉴...우리집 풀무원사랑💚..#풀무원바른먹거리#푸드머스#스틱피쉬커틀렛,"[저녁메뉴, 풀무원바른먹거리, 푸드머스, 스틱피쉬커틀렛]"
2,💕풀무원 바른 먹거리 온라인 비대면 교육💕1차시/2월17일2차시/2월24일풀무원 바...,"[풀무원, 풀무원바른먹거리, 바른먹거리교육, 비대면교육, 초등집콕생활]"
3,..풀무원 달걀농장 체험중~😘..#오한결#풀무원바른먹거리 #키자니아서울 #달걀농장체...,"[오한결, 풀무원바른먹거리, 키자니아서울, 달걀농장체험중, 친환경농부가되어보아요, ..."
4,#딸기야🍓🍓🍓🍓🍓⁣⁣출동! 딸기밭으로~~~~⁣친구들이 너무나 기다렸던⁣노랑차 타는 ...,"[딸기야, 딸기밭견학, 우리밖에없어요, 오늘도, 행복미소최고]"
5,#오늘의간식🍈🌭⠀⠀달콤한 메론과 우리밀 핫도그💜⠀초록야채.과일이 든🧃⠀유산균 음료 ...,"[오늘의간식, 행복미소어린이집, 맛있는간식, 무럭무럭자라요, 비오는날, 코코낸내, ..."
6,#친구들모여라👶🏻👧🏻🧒🏻⁣⁣행복미소 어린이집에서⁣2021년도 신입원아를 모집합니다⁣...,"[친구들모여라, 포항, 연일, 가정어린이집, 49평, 11월, 가을, 효자, 유강,..."
7,#풀무원 #풀무원교육 #풀무원바른먹거리 #풀무원바른먹거리교육 #바른먹거리교육 #초등...,"[풀무원, 풀무원교육, 풀무원바른먹거리, 풀무원바른먹거리교육, 바른먹거리교육, 초등..."
8,#21년새학기이야기⁣⁣3월👶🏻🧒👧⁣신학기 시작한지 벌써 3주차❤⁣⁣우리 예쁜반.햇님...,"[21년새학기이야기, 바른먹거리, 행복미소어린이집, 내일은3월생일파티, 친구들아, ..."
9,"오수휴게소의 따끈한 ⚡️신제품⚡️대한민국 대표 여름 과일 중 하나, 경북 성주에서 ...",[]


In [14]:
df.to_csv("인스타.csv", header = True, index = False, encoding='utf-8-sig')